In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer, StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf
from sklearn.linear_model import LogisticRegression

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from google.colab import files
uploaded = files.upload()

Saving Data.csv to Data.csv
Saving Test_set_values.csv to Test_set_values.csv
Saving Training_set_labels.csv to Training_set_labels.csv


In [0]:
import io

In [128]:
data = pd.read_csv(io.StringIO(uploaded['Data.csv'].decode('utf-8')))
labels = pd.read_csv(io.StringIO(uploaded['Training_set_labels.csv'].decode('utf-8')))
test_data = pd.read_csv(io.StringIO(uploaded['Test_set_values.csv'].decode('utf-8')))
merged_data = pd.concat([data, test_data])
data_len = len(data)
print('Data Length: ',data_len)
test_data_len = len(test_data)
print('Test Data Length: ', test_data_len)
merged_data_len = len(merged_data)
print('Merged data Length: ',merged_data_len)
merged_data.head()

Data Length:  59400
Test Data Length:  14850
Merged data Length:  74250


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [129]:
merged_data.isnull().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    4504
gps_height                   0
installer                 4532
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 470
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            4155
recorded_by                  0
scheme_management         4846
scheme_name              35258
permit                    3793
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [0]:
# Drop Schem_name feature..... Too Many Null Values
mergerd_data = merged_data.drop(['scheme_name'], axis = 1)

In [131]:
'''     
count = 0
for col in merged_data['amount_tsh']:
    if col == 0:
        count+=1
print(count)        
'''        

"     \ncount = 0\nfor col in merged_data['amount_tsh']:\n    if col == 0:\n        count+=1\nprint(count)        \n"

In [0]:
#drop funder.. Too many Unique Values
#drop public meeting, and permint columns
# Drop Schem_name feature..... Too Many Null Values

merged_data = merged_data.drop(['funder','amount_tsh','subvillage','installer','public_meeting', 'permit','scheme_name','payment','extraction_type','extraction_type_group','quality_group','quantity_group','source_type','waterpoint_type_group'], axis = 1)


In [133]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74250 entries, 0 to 14849
Data columns (total 26 columns):
id                       74250 non-null int64
date_recorded            74250 non-null object
gps_height               74250 non-null int64
longitude                74250 non-null float64
latitude                 74250 non-null float64
wpt_name                 74250 non-null object
num_private              74250 non-null int64
basin                    74250 non-null object
region                   74250 non-null object
region_code              74250 non-null int64
district_code            74250 non-null int64
lga                      74250 non-null object
ward                     74250 non-null object
population               74250 non-null int64
recorded_by              74250 non-null object
scheme_management        69404 non-null object
construction_year        74250 non-null int64
extraction_type_class    74250 non-null object
management               74250 non-null object
man

In [134]:
merged_data['wpt_name'].value_counts()

none                       4440
Shuleni                    2183
Zahanati                   1034
Msikitini                   647
Kanisani                    390
Sokoni                      328
Bombani                     323
Ofisini                     321
School                      260
Shule Ya Msingi             247
Shule                       191
Sekondari                   178
Muungano                    159
Mkombozi                    145
Upendo                      130
Madukani                    128
Kituo Cha Afya              113
Mbugani                     113
Umoja                       105
Mkuyuni                     103
Center                      100
Hospital                    100
Kisimani                     97
Ccm                          95
Mtakuja                      91
Songambele                   90
Ofisi Ya Kijiji              89
Bwawani                      86
Tankini                      82
Bondeni                      80
                           ... 
Isangati

In [0]:
merged_data.drop(['wpt_name'], axis = 1, inplace =True)

In [136]:
merged_data['date_recorded'].value_counts()

2011-03-17    695
2011-03-15    694
2013-02-03    684
2011-03-16    650
2011-03-14    649
2011-03-18    627
2011-03-19    576
2013-02-04    575
2013-01-29    570
2011-03-04    562
2013-02-14    550
2013-01-24    546
2013-01-30    533
2011-03-05    532
2011-03-09    522
2011-03-11    522
2011-03-23    520
2013-03-15    516
2013-02-15    516
2013-01-18    513
2013-02-16    505
2013-01-23    498
2011-03-12    493
2011-03-24    490
2011-03-30    486
2013-03-19    486
2011-03-03    478
2013-02-26    475
2013-02-18    473
2013-02-13    470
             ... 
2011-09-18      1
2011-09-07      1
2011-09-10      1
2004-01-12      1
2011-08-31      1
2004-01-09      1
2011-09-06      1
2011-09-28      1
2011-09-01      1
2012-01-25      1
2011-09-11      1
2011-09-09      1
2002-10-14      1
2011-09-22      1
2004-01-04      1
2011-09-27      1
2011-09-05      1
2004-04-01      1
2011-09-19      1
2004-12-03      1
2011-08-29      1
2012-01-21      1
2011-08-30      1
2011-09-20      1
2011-09-26

In [0]:
merged_data.drop(['date_recorded', 'construction_year','lga','ward','recorded_by','longitude','latitude'], axis = 1, inplace = True)

In [138]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74250 entries, 0 to 14849
Data columns (total 18 columns):
id                       74250 non-null int64
gps_height               74250 non-null int64
num_private              74250 non-null int64
basin                    74250 non-null object
region                   74250 non-null object
region_code              74250 non-null int64
district_code            74250 non-null int64
population               74250 non-null int64
scheme_management        69404 non-null object
extraction_type_class    74250 non-null object
management               74250 non-null object
management_group         74250 non-null object
payment_type             74250 non-null object
water_quality            74250 non-null object
quantity                 74250 non-null object
source                   74250 non-null object
source_class             74250 non-null object
waterpoint_type          74250 non-null object
dtypes: int64(6), object(12)
memory usage: 10.8+ MB

In [139]:
cleaned_train_data = merged_data[:data_len]
cleaned_test_data = merged_data[data_len:]
print('data',data_len)
print('cleaned_train_data',len(cleaned_train_data))
print('test data', test_data_len)
print('cleaned_test_data',len(cleaned_test_data))
cleaned_train_data.to_csv('Cleaned Train Data.csv', index = False)
cleaned_test_data.to_csv('Cleaned Test Data.csv', index =False)

data 59400
cleaned_train_data 59400
test data 14850
cleaned_test_data 14850


In [140]:
merged_data.shape

(74250, 18)

In [0]:
#Collect Categorical Data
cat_data = merged_data.select_dtypes(include=['object'])

In [142]:
cat_data.shape

(74250, 12)

In [143]:
#Collect Numerical Data

num_data = merged_data.drop(cat_data, axis = 1)
num_data.shape

(74250, 6)

In [144]:
num_data.head()

,id,gps_height,num_private,region_code,district_code,population
0,69572,1390,0,11,5,109
1,8776,1399,0,20,2,280
2,34310,686,0,21,4,250
3,67743,263,0,90,63,58
4,19728,0,0,18,1,0


In [0]:
dummy = pd.get_dummies(cat_data)

In [0]:
X = pd.concat([num_data, dummy], axis = 1)

In [147]:
X.isnull().sum()

X = np.asarray(X)

sc = StandardScaler()

X = sc.fit_transform(X)
X

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[ 1.51382407,  1.0455807 , -0.04007089, ..., -0.64645553,
        -0.11438893, -0.34774116],
       [-1.322587  ,  1.05857228, -0.04007089, ..., -0.64645553,
        -0.11438893, -0.34774116],
       [-0.13130928,  0.02935041, -0.04007089, ..., -0.64645553,
        -0.11438893, -0.34774116],
       ...,
       [-0.39075533,  1.16972247, -0.04007089, ..., -0.64645553,
        -0.11438893, -0.34774116],
       [-0.16947272,  0.4797252 , -0.04007089, ..., -0.64645553,
        -0.11438893, -0.34774116],
       [ 1.47346787, -0.26656891, -0.04007089, ..., -0.64645553,
        -0.11438893, -0.34774116]])

In [0]:
train_data= X[:data_len]
test_data = X[data_len:]

In [149]:

label = labels.drop(['id'], axis = 1)

encoder = LabelBinarizer()

labelT = encoder.fit_transform(label)

#label_dummy = pd.get_dummies(label)

#value = {'functional':2, 'functional needs repair':1,
#                   'non functional':0}

#label['status_group']  = label.status_group.replace(value)
#label_dummy.shape

labelT

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]])

In [150]:
n_dim = train_data.shape[1]
print('input size: ',n_dim)
n_class = labelT.shape[1]
print('Classes: ',n_class)
print(train_data.shape)
print(labelT.shape)

print("X Shape =", train_data.shape, "Y Shape =",labelT.shape, "m = ",n_dim, "n classes found in test data=", n_class)

input size:  112
Classes:  3
(59400, 112)
(59400, 3)
X Shape = (59400, 112) Y Shape = (59400, 3) m =  112 n classes found in test data= 3


In [0]:
learning_rate = 0.01
epochs = 1000

layer_1_nodes = 100
layer_2_nodes = 100
layer_3_nodes = 100


x = tf.placeholder(tf.float32, [None, n_dim])
y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.zeros([n_dim,n_class]))
b = tf.Variable(tf.zeros([n_class]))

In [0]:
#create weights and biases

weights = {'w1': tf.Variable(tf.random_normal([n_dim, layer_1_nodes])),
          'w2': tf.Variable(tf.random_normal([layer_1_nodes, layer_2_nodes])),
          'w3': tf.Variable(tf.random_normal([layer_2_nodes, layer_3_nodes])),
          'wout': tf.Variable(tf.random_normal([layer_3_nodes, n_class])),}
          

biases = {'b1': tf.Variable(tf.random_normal([layer_1_nodes])),
         'b2': tf.Variable(tf.random_normal([layer_2_nodes])),
         'b3': tf.Variable(tf.random_normal([layer_3_nodes])),
         'bout': tf.Variable(tf.random_normal([n_class])),
         }

In [0]:
def Neural_Netwrk_model(x, weights, biases):
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)
    output = tf.matmul(layer_3, weights['wout']) + biases['bout']
    
    return output

In [162]:
'''
def One_Hot_Encoding(y):
    
    total_rows = len(y)
    print(total_rows)
    unique_classes = len(np.unique(y))
    print(unique_classes)
    One_Hot_Encoding = np.zeros((total_rows,unique_classes))
    
    One_Hot_Encoding[np.arange(total_rows),y]=1
    
    return One_Hot_Encoding

    
encoder = LabelEncoder()
encoder.fit(label)
label =encoder.transform(label)

labels = One_Hot_Encoding(label)
labels
'''

'\ndef One_Hot_Encoding(y):\n    \n    total_rows = len(y)\n    print(total_rows)\n    unique_classes = len(np.unique(y))\n    print(unique_classes)\n    One_Hot_Encoding = np.zeros((total_rows,unique_classes))\n    \n    One_Hot_Encoding[np.arange(total_rows),y]=1\n    \n    return One_Hot_Encoding\n\n    \nencoder = LabelEncoder()\nencoder.fit(label)\nlabel =encoder.transform(label)\n\nlabels = One_Hot_Encoding(label)\nlabels\n'

In [0]:
x_train, x_test, y_train, y_test = train_test_split(train_data, labelT, test_size= 0.20, random_state = 0)

In [0]:
model = Neural_Netwrk_model(x, weights, biases)
accuracy_history = []
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [0]:
#cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(model), reduction_indices=[1]))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)


In [166]:
for epoch in range(epochs):
    
    sess.run(optimizer, feed_dict={x:x_train, y:y_train})
    c = sess.run(cost, feed_dict={x:x_train, y:y_train})
    
    correct_prediction = tf.equal(tf.argmax(model,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))   
    accuracy_t = (sess.run(accuracy,feed_dict={x:x_train, y:y_train}))
    accuracy_history.append(accuracy)
    
    print('completed: ',epoch,'eopch out of: ',epochs,'cost is : ',c,'accuracy: ', accuracy_t)
      

completed:  0 eopch out of:  1000 cost is :  39910.176 accuracy:  0.38415405
completed:  1 eopch out of:  1000 cost is :  1654.6 accuracy:  0.5173611
completed:  2 eopch out of:  1000 cost is :  1306.0719 accuracy:  0.33468014
completed:  3 eopch out of:  1000 cost is :  599.51825 accuracy:  0.5179714
completed:  4 eopch out of:  1000 cost is :  207.69456 accuracy:  0.34736952
completed:  5 eopch out of:  1000 cost is :  136.80795 accuracy:  0.5002946
completed:  6 eopch out of:  1000 cost is :  114.96513 accuracy:  0.49747476
completed:  7 eopch out of:  1000 cost is :  100.14384 accuracy:  0.49987373
completed:  8 eopch out of:  1000 cost is :  89.67875 accuracy:  0.4959596
completed:  9 eopch out of:  1000 cost is :  84.60208 accuracy:  0.5077231
completed:  10 eopch out of:  1000 cost is :  89.21493 accuracy:  0.46879208
completed:  11 eopch out of:  1000 cost is :  105.81483 accuracy:  0.5254209
completed:  12 eopch out of:  1000 cost is :  82.92912 accuracy:  0.44612795
completed

completed:  50 eopch out of:  1000 cost is :  10.914864 accuracy:  0.4787037
completed:  51 eopch out of:  1000 cost is :  10.604251 accuracy:  0.5153199
completed:  52 eopch out of:  1000 cost is :  10.12 accuracy:  0.47725168
completed:  53 eopch out of:  1000 cost is :  9.88507 accuracy:  0.51662457
completed:  54 eopch out of:  1000 cost is :  9.382449 accuracy:  0.4785143
completed:  55 eopch out of:  1000 cost is :  9.209306 accuracy:  0.5184554
completed:  56 eopch out of:  1000 cost is :  8.713207 accuracy:  0.47984007
completed:  57 eopch out of:  1000 cost is :  8.599501 accuracy:  0.5203914
completed:  58 eopch out of:  1000 cost is :  8.103932 accuracy:  0.48013467
completed:  59 eopch out of:  1000 cost is :  8.045025 accuracy:  0.5217593
completed:  60 eopch out of:  1000 cost is :  7.5719604 accuracy:  0.48089224
completed:  61 eopch out of:  1000 cost is :  7.5480204 accuracy:  0.52285355
completed:  62 eopch out of:  1000 cost is :  7.0997777 accuracy:  0.48085016
comp

completed:  100 eopch out of:  1000 cost is :  3.090541 accuracy:  0.49577022
completed:  101 eopch out of:  1000 cost is :  3.1521869 accuracy:  0.5337332
completed:  102 eopch out of:  1000 cost is :  3.0051072 accuracy:  0.49640152
completed:  103 eopch out of:  1000 cost is :  3.06695 accuracy:  0.5337542
completed:  104 eopch out of:  1000 cost is :  2.9246817 accuracy:  0.49701178
completed:  105 eopch out of:  1000 cost is :  2.9864602 accuracy:  0.5342172
completed:  106 eopch out of:  1000 cost is :  2.8504748 accuracy:  0.49781144
completed:  107 eopch out of:  1000 cost is :  2.9123132 accuracy:  0.53440654
completed:  108 eopch out of:  1000 cost is :  2.7820244 accuracy:  0.49825338
completed:  109 eopch out of:  1000 cost is :  2.843802 accuracy:  0.53470117
completed:  110 eopch out of:  1000 cost is :  2.7171934 accuracy:  0.49911615
completed:  111 eopch out of:  1000 cost is :  2.7767003 accuracy:  0.53470117
completed:  112 eopch out of:  1000 cost is :  2.655817 acc

completed:  150 eopch out of:  1000 cost is :  1.9534124 accuracy:  0.50934345
completed:  151 eopch out of:  1000 cost is :  1.9781636 accuracy:  0.53781563
completed:  152 eopch out of:  1000 cost is :  1.9294823 accuracy:  0.5098695
completed:  153 eopch out of:  1000 cost is :  1.9533963 accuracy:  0.53815234
completed:  154 eopch out of:  1000 cost is :  1.9075843 accuracy:  0.5099958
completed:  155 eopch out of:  1000 cost is :  1.9301513 accuracy:  0.5382155
completed:  156 eopch out of:  1000 cost is :  1.8864498 accuracy:  0.5103535
completed:  157 eopch out of:  1000 cost is :  1.9078974 accuracy:  0.53834176
completed:  158 eopch out of:  1000 cost is :  1.8660539 accuracy:  0.5106902
completed:  159 eopch out of:  1000 cost is :  1.886576 accuracy:  0.5385311
completed:  160 eopch out of:  1000 cost is :  1.8464959 accuracy:  0.51087964
completed:  161 eopch out of:  1000 cost is :  1.8660018 accuracy:  0.53872055
completed:  162 eopch out of:  1000 cost is :  1.8278965 ac

completed:  200 eopch out of:  1000 cost is :  1.563633 accuracy:  0.5146465
completed:  201 eopch out of:  1000 cost is :  1.5725865 accuracy:  0.5405093
completed:  202 eopch out of:  1000 cost is :  1.5539231 accuracy:  0.51496214
completed:  203 eopch out of:  1000 cost is :  1.562744 accuracy:  0.54046714
completed:  204 eopch out of:  1000 cost is :  1.5445285 accuracy:  0.5148569
completed:  205 eopch out of:  1000 cost is :  1.5532498 accuracy:  0.5406355
completed:  206 eopch out of:  1000 cost is :  1.5355192 accuracy:  0.51506734
completed:  207 eopch out of:  1000 cost is :  1.5440329 accuracy:  0.5406776
completed:  208 eopch out of:  1000 cost is :  1.5267272 accuracy:  0.5151515
completed:  209 eopch out of:  1000 cost is :  1.5348498 accuracy:  0.54084593
completed:  210 eopch out of:  1000 cost is :  1.5181584 accuracy:  0.5152778
completed:  211 eopch out of:  1000 cost is :  1.5259457 accuracy:  0.54097223
completed:  212 eopch out of:  1000 cost is :  1.5098774 accu

...

completed:  796 eopch out of:  1000 cost is :  0.9436829 accuracy:  0.54246634
completed:  797 eopch out of:  1000 cost is :  0.944912 accuracy:  0.5478535
completed:  798 eopch out of:  1000 cost is :  0.94328254 accuracy:  0.5424874
completed:  799 eopch out of:  1000 cost is :  0.9445081 accuracy:  0.5479377
completed:  800 eopch out of:  1000 cost is :  0.9428849 accuracy:  0.5425505
completed:  801 eopch out of:  1000 cost is :  0.9441078 accuracy:  0.54791665
completed:  802 eopch out of:  1000 cost is :  0.9424915 accuracy:  0.54257154
completed:  803 eopch out of:  1000 cost is :  0.9437115 accuracy:  0.54791665
completed:  804 eopch out of:  1000 cost is :  0.9421023 accuracy:  0.5426978
completed:  805 eopch out of:  1000 cost is :  0.9433195 accuracy:  0.54787457
completed:  806 eopch out of:  1000 cost is :  0.941716 accuracy:  0.54267675
completed:  807 eopch out of:  1000 cost is :  0.9429297 accuracy:  0.54791665
completed:  808 eopch out of:  1000 cost is :  0.94133186 

completed:  845 eopch out of:  1000 cost is :  0.9360258 accuracy:  0.54852694
completed:  846 eopch out of:  1000 cost is :  0.9345534 accuracy:  0.5434764
completed:  847 eopch out of:  1000 cost is :  0.935688 accuracy:  0.548548
completed:  848 eopch out of:  1000 cost is :  0.9342225 accuracy:  0.5434975
completed:  849 eopch out of:  1000 cost is :  0.9353533 accuracy:  0.54859006
completed:  850 eopch out of:  1000 cost is :  0.9338971 accuracy:  0.5434764
completed:  851 eopch out of:  1000 cost is :  0.9350244 accuracy:  0.54863214
completed:  852 eopch out of:  1000 cost is :  0.93357426 accuracy:  0.5435396
completed:  853 eopch out of:  1000 cost is :  0.93469614 accuracy:  0.54863214
completed:  854 eopch out of:  1000 cost is :  0.9332534 accuracy:  0.5436448
completed:  855 eopch out of:  1000 cost is :  0.934371 accuracy:  0.5486532
completed:  856 eopch out of:  1000 cost is :  0.93293566 accuracy:  0.54372895
completed:  857 eopch out of:  1000 cost is :  0.93404835 a

completed:  895 eopch out of:  1000 cost is :  0.9283555 accuracy:  0.5493266
completed:  896 eopch out of:  1000 cost is :  0.92706376 accuracy:  0.5441498
completed:  897 eopch out of:  1000 cost is :  0.9280768 accuracy:  0.54934764
completed:  898 eopch out of:  1000 cost is :  0.926792 accuracy:  0.54421294
completed:  899 eopch out of:  1000 cost is :  0.92779964 accuracy:  0.5493687
completed:  900 eopch out of:  1000 cost is :  0.9265219 accuracy:  0.5442761
completed:  901 eopch out of:  1000 cost is :  0.92752445 accuracy:  0.5493897
completed:  902 eopch out of:  1000 cost is :  0.92625356 accuracy:  0.5442551
completed:  903 eopch out of:  1000 cost is :  0.92725086 accuracy:  0.5494949
completed:  904 eopch out of:  1000 cost is :  0.9259865 accuracy:  0.54429716
completed:  905 eopch out of:  1000 cost is :  0.92697847 accuracy:  0.549516
completed:  906 eopch out of:  1000 cost is :  0.9257212 accuracy:  0.5442761
completed:  907 eopch out of:  1000 cost is :  0.9267078 

completed:  944 eopch out of:  1000 cost is :  0.92099065 accuracy:  0.5447601
completed:  945 eopch out of:  1000 cost is :  0.9218822 accuracy:  0.55006313
completed:  946 eopch out of:  1000 cost is :  0.9207594 accuracy:  0.54478115
completed:  947 eopch out of:  1000 cost is :  0.9216469 accuracy:  0.5500842
completed:  948 eopch out of:  1000 cost is :  0.92053086 accuracy:  0.54484427
completed:  949 eopch out of:  1000 cost is :  0.9214142 accuracy:  0.55012625
completed:  950 eopch out of:  1000 cost is :  0.92030364 accuracy:  0.54488635
completed:  951 eopch out of:  1000 cost is :  0.92118204 accuracy:  0.5502736
completed:  952 eopch out of:  1000 cost is :  0.9200771 accuracy:  0.54499155
completed:  953 eopch out of:  1000 cost is :  0.9209507 accuracy:  0.5502525
completed:  954 eopch out of:  1000 cost is :  0.9198523 accuracy:  0.5450758
completed:  955 eopch out of:  1000 cost is :  0.9207213 accuracy:  0.55029464
completed:  956 eopch out of:  1000 cost is :  0.9196

completed:  993 eopch out of:  1000 cost is :  0.91667765 accuracy:  0.5505892
completed:  994 eopch out of:  1000 cost is :  0.91570646 accuracy:  0.5460648
completed:  995 eopch out of:  1000 cost is :  0.9164891 accuracy:  0.5505892
completed:  996 eopch out of:  1000 cost is :  0.9155226 accuracy:  0.546128
completed:  997 eopch out of:  1000 cost is :  0.9163008 accuracy:  0.5506734
completed:  998 eopch out of:  1000 cost is :  0.9153393 accuracy:  0.546128
completed:  999 eopch out of:  1000 cost is :  0.9161139 accuracy:  0.5506313


In [176]:
prediction=tf.argmax(model,1)
tf = prediction.eval(feed_dict={x: test_data}, session=sess)
print ("predictions", tf)

predictions [0 0 0 ... 0 0 0]


In [0]:
#Create submission file

submit = pd.DataFrame({'ID': test_data['id'],
                       'status_group': tf
                      })
  
value = {2:'functional', 1:'functional needs repair',
                   0:'non functional'}

submit['status_group']  = label.status_group.replace(value)       

submit.to_csv('submission.csv', index=False)